In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
    ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
    ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
    ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4444


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(11, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0050
Function value obtained: 0.4723
Current minimum: 0.4723
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0024
Function value obtained: 0.4721
Current minimum: 0.4721
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0025
Function value obtained: 0.4728
Current minimum: 0.4721
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0023
Function value obtained: 0.4727
Current minimum: 0.4721
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.5047
Function value obtained: 0.4714
Current minimum: 0.4714
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.5063
Function value obtained: 0.4717
Current minimum: 0.4714
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.4951
Function value obtained: 0.4719
Current minimum: 0.4714
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.4956
Function value obtained: 0.4722
Current minimum: 0.4714
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.4710
Function value obtained: 0.4727
Current minimum: 0.4714
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.6901
Function value obtained: 0.4726
Current minimum: 0.4713
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.7004
Function value obtained: 0.4727
Current minimum: 0.4713
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.7094
Function value obtained: 0.4728
Current minimum: 0.4713
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.7529
Function value obtained: 0.4736
Current minimum: 0.4713
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.7741
Function value obtained: 0.4713
Current minimum: 0.4713
Iteration No: 86 started. Sea

Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 0.2420
Function value obtained: 0.4387
Current minimum: 0.4371
Iteration No: 21 started. Searching for the next optimal point.
Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.3381
Function value obtained: 0.4372
Current minimum: 0.4371
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2555
Function value obtained: 0.4378
Current minimum: 0.4371
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.3317
Function value obtained: 0.4373
Current minimum: 0.4371
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3376
Function value obtained: 0.4370
Current minimum: 0.4370
Iteration No: 25 started. Sea

Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 0.5563
Function value obtained: 0.4386
Current minimum: 0.4368
Iteration No: 61 started. Searching for the next optimal point.
Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.5471
Function value obtained: 0.4369
Current minimum: 0.4368
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.5644
Function value obtained: 0.4380
Current minimum: 0.4368
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.5411
Function value obtained: 0.4378
Current minimum: 0.4368
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.6246
Function value obtained: 0.4382
Current minimum: 0.4368
Iteration No: 65 started. Sea

...

Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 0.4125
Function value obtained: 0.4647
Current minimum: 0.4633
Iteration No: 38 started. Searching for the next optimal point.
Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 0.4468
Function value obtained: 0.4646
Current minimum: 0.4633
Iteration No: 39 started. Searching for the next optimal point.
Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 0.4031
Function value obtained: 0.4653
Current minimum: 0.4633
Iteration No: 40 started. Searching for the next optimal point.
Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 0.4877
Function value obtained: 0.4667
Current minimum: 0.4633
Iteration No: 41 started. Searching for the next optimal point.
Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.4079
Function value obtained: 0.4636
Current minimum: 0.4633
Iteration No: 42 started. Sea

Iteration No: 77 ended. Search finished for the next optimal point.
Time taken: 0.6380
Function value obtained: 0.4652
Current minimum: 0.4630
Iteration No: 78 started. Searching for the next optimal point.
Iteration No: 78 ended. Search finished for the next optimal point.
Time taken: 0.6291
Function value obtained: 0.4648
Current minimum: 0.4630
Iteration No: 79 started. Searching for the next optimal point.
Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 0.6989
Function value obtained: 0.4659
Current minimum: 0.4630
Iteration No: 80 started. Searching for the next optimal point.
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 0.7022
Function value obtained: 0.4630
Current minimum: 0.4630
Iteration No: 81 started. Searching for the next optimal point.
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.6962
Function value obtained: 0.4646
Current minimum: 0.4630
Iteration No: 82 started. Sea

Iteration No: 16 ended. Search finished for the next optimal point.
Time taken: 0.2468
Function value obtained: 0.4383
Current minimum: 0.4374
Iteration No: 17 started. Searching for the next optimal point.
Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 0.2549
Function value obtained: 0.4379
Current minimum: 0.4374
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 0.2409
Function value obtained: 0.4380
Current minimum: 0.4374
Iteration No: 19 started. Searching for the next optimal point.
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 0.3041
Function value obtained: 0.4386
Current minimum: 0.4374
Iteration No: 20 started. Searching for the next optimal point.
Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 0.2666
Function value obtained: 0.4384
Current minimum: 0.4374
Iteration No: 21 started. Sea

Iteration No: 56 ended. Search finished for the next optimal point.
Time taken: 0.6484
Function value obtained: 0.4377
Current minimum: 0.4373
Iteration No: 57 started. Searching for the next optimal point.
Iteration No: 57 ended. Search finished for the next optimal point.
Time taken: 0.6634
Function value obtained: 0.4385
Current minimum: 0.4373
Iteration No: 58 started. Searching for the next optimal point.
Iteration No: 58 ended. Search finished for the next optimal point.
Time taken: 0.7431
Function value obtained: 0.4380
Current minimum: 0.4373
Iteration No: 59 started. Searching for the next optimal point.
Iteration No: 59 ended. Search finished for the next optimal point.
Time taken: 0.6421
Function value obtained: 0.4377
Current minimum: 0.4373
Iteration No: 60 started. Searching for the next optimal point.
Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 0.5776
Function value obtained: 0.4384
Current minimum: 0.4373
Iteration No: 61 started. Sea

Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 0.8828
Function value obtained: 0.4379
Current minimum: 0.4371
Iteration No: 97 started. Searching for the next optimal point.
Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 0.9560
Function value obtained: 0.4376
Current minimum: 0.4371
Iteration No: 98 started. Searching for the next optimal point.
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 0.8930
Function value obtained: 0.4371
Current minimum: 0.4371
Iteration No: 99 started. Searching for the next optimal point.
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 0.9658
Function value obtained: 0.4381
Current minimum: 0.4371
Iteration No: 100 started. Searching for the next optimal point.
Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 0.9567
Function value obtained: 0.4371
Current minimum: 0.4371
[0.09255323 0.01623098 0.01

In [10]:
best_weights

{'cohesion': array([0.10470163, 0.05846104, 0.0156783 , 0.10620709, 0.0156783 ,
        0.15678298, 0.04473068, 0.05607159, 0.12812244, 0.15678298,
        0.15678298]),
 'syntax': array([0.12165438, 0.01907494, 0.05313258, 0.19074942, 0.01907494,
        0.19074942, 0.03864318, 0.13175623, 0.12788561, 0.01907494,
        0.08820435]),
 'vocabulary': array([0.24234155, 0.02423416, 0.02423416, 0.24234155, 0.09275811,
        0.10934031, 0.03624484, 0.07077628, 0.10926074, 0.02423416,
        0.02423416]),
 'phraseology': array([0.13127692, 0.0158882 , 0.0158882 , 0.158882  , 0.0330158 ,
        0.158882  , 0.0158882 , 0.0658782 , 0.158882  , 0.08663646,
        0.158882  ]),
 'grammar': array([0.14646024, 0.02969522, 0.02969522, 0.29695224, 0.02969522,
        0.02969522, 0.06408182, 0.02969522, 0.12606321, 0.02969522,
        0.18827116]),
 'conventions': array([0.09255323, 0.01623098, 0.01623098, 0.13503673, 0.08439165,
        0.1200089 , 0.05291859, 0.09047877, 0.16230975, 0.0675306

In [11]:
scores

{'cohesion': 0.4712210056276879,
 'syntax': 0.4366810482646107,
 'vocabulary': 0.4047566507653655,
 'phraseology': 0.4460306509766572,
 'grammar': 0.46292908766517415,
 'conventions': 0.4371392602255078}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4431


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.945369,0.913558,0.910709,0.824646,0.886653
syntax,0.945369,1.000000,0.937019,0.960221,0.916342,0.928913
vocabulary,0.913558,0.937019,1.000000,0.958414,0.874691,0.891136
phraseology,0.910709,0.960221,0.958414,1.000000,0.942280,0.881344
grammar,0.824646,0.916342,0.874691,0.942280,1.000000,0.855159
conventions,0.886653,0.928913,0.891136,0.881344,0.855159,1.000000
